# Automatic Target Parameter Calculation

In general, passive bistatic radars are able to measure the bistatic range, and the bistatic Doppler shift of the  reflected signals. In case the radar utilize phased array antenna system, the measurement of the azimuth and elevation angles are also become possible. Of course the direct calculation of these quantities is rather straightforward. There exists simple formulas, which are widely available in the literature [Willis2005].  

In this report I am going to summarize the basic equations, that could be used to determine these quantities from standard geodetic data. Beside this I am introducing a simple python based script which implements these calculations. This function could be usefull at many points of a system development. To check the basic operation of a syetem, to estimate its accuracy, to implement automatic performance evaluation environments based on real measurmemts, to automatically generate reference data for algorithm tuning and even to automatically generate large amount of training data from measurement to train AI algorithms. 

Examples for all these applications will be presented in the second part of this report. 

## Bistatic range calculation: 

Bistatic range is the path difference of the reflected signal and the direct signal. It can be calculated with the following formula 
\begin{equation}
R_b = R_r + R_t - L
\end{equation}
where $R_t$ and $R_r$ are the transmitter to target and the receiver to target distances respectively. $L$ denotes the baseline distance, which is the illuminator to receiver distance. The bistatic distance has major importance, as the radar can primary determine this quantity from the measurement. From a range-Doppler map (cross-correlation detector output) one can measure the time delay between the direct path signal and target reflection by locating the correlation peak corresponding to the target.  
In case the difference in range cells is K, then the bistatic range is simply: 
\begin{equation}
R_b = Kc/fs,
\end{equation}
where $c$ is the speed of light and $f_s$ is the sampling frequency. In the opposite case, if we know the exact location of the IoO, the receiver and the target we can calculate the observable bistatic range. Let us assume, that all the coordinates are in lattitude, longitude, elevation format. To calculate distances (and later projections for the Doppler frequency) we can convert these coordinates into a 3D cartesian coordinate system. The forward conversion is rather straightforward, but to use Datums as well like WGS84 I have utilised the ECEF converter of the Pygeodesy module [Brouwers2020]. 

In [1]:
import numpy as np
from pygeodesy import ecef
from pygeodesy import Datums

# Example coordinates : Budapest, Heroes' Square
target_lat = 47.515136 # [deg]
target_lon = 19.078176 # [deg]
target_ele = 108       # [m]

ecef_converter = ecef.EcefYou(Datums.Sphere)
target_ecef_raw = ecef_converter.forward(target_lat, target_lon, target_ele)
target_ecef     =np.array([target_ecef_raw[0], target_ecef_raw[1], target_ecef_raw[2]])

print("Converted X, Y Z coordinates", target_ecef)

Converted X, Y Z coordinates [4066672.98023294 1406477.26777827 4698416.91233034]


Knowing all the 3D coordinates, we can simply calculate the distances. Let us define the following vectors:

- Illuminator:
$\textbf{i} = \left(x_i,y_i,z_i\right)$
- Radar:
$\textbf{r} = \left(x_r,y_r,z_r\right)$ 
- Target:
$\textbf{t} = \left(x_t,y_t,z_t\right)$

With this notation the distances are:
$Rt = |\textbf{i}-\textbf{t}| $,
$Rr = |\textbf{t}-\textbf{r}| $ and 
$L = |\textbf{i}-\textbf{r}| $, and the observable sample delay is: 
\begin{equation}
\tau = \frac{(Rt+Rr)-L}{c T_s} ,
\end{equation}
where $T_s = 1/f_s$ is the sampling period.


In [2]:
# Example coordinates of the IoO : Budapest, Széchenyi hill
ioo_lat = 47.49166667  # [deg]
ioo_lon = 18.97888889  # [deg]
ioo_ele = 457 + 182    # [m] Above Sea Level + Above Ground Level

# Convert the geodetic IoO coordinates to ECEF coordinates
ioo_ecef = ecef_converter.forward(ioo_lat, ioo_lon, ioo_ele)
ioo_ecef = np.array([ioo_ecef[0],ioo_ecef[1], ioo_ecef[2]])

# Example coordinates for the radar: Budapest, Mathhias Church
radar_lat = 47.501807 # [deg]
radar_lon = 19.034006 # [deg]
radar_ele = 169       # [m]

# Convert geodetic radar position to ECEF coordinates
radar_ecef = ecef_converter.forward(radar_lat, radar_lon, radar_ele)
radar_ecef = np.array([radar_ecef[0],radar_ecef[1], radar_ecef[2]])

# Baseline distance
L = np.sqrt(np.sum(np.abs(radar_ecef-ioo_ecef)**2)) 

 # Target to IoO distance
Rt = np.sqrt(np.sum(np.abs(target_ecef-ioo_ecef)**2)) 

# Target to radar distance
Rr = np.sqrt(np.sum(np.abs(target_ecef-radar_ecef)**2)) 

# Bistatic distance
Rb = Rt+Rr-L

# Example sampling frequency is 10 MHz
fs = 10 * 10**6

# Speed of light
c= 3*10**8

# Observable time delay in number of samples
tau = Rb*fs/(c)

print("Baseline distance: {:.1f} m".format(L))
print("IoO to target distance: {:.1f} m".format(Rt))
print("Target to radar distance: {:.1f} m".format(Rr))
print("Bistatic distance: {:.1f} m".format(Rb))
print("Observable time delay: {:.1f} sample".format(tau))

Baseline distance: 4317.5 m
IoO to target distance: 7919.9 m
Target to radar distance: 3634.2 m
Bistatic distance: 7236.6 m
Observable time delay: 241.2 sample


## Bistatic Doppler calculation 

To calculate the bistatic Doppler frequency shift we also need the speed and the moving direction of the target. 
Let us use the $\textbf{v}$ vector to describe both these quantities. $|\textbf{v}|$ gives us the absolute moving speed, while $\textbf{v}/|\textbf{v}|$ gives us the normalized direction vector. We can determine the estimation of this speed vector in the 3D space from the lattitude, longitude, speed and direction values with the next few calculation steps: 

- Calculate the lat, long coordinates of the target with some distances away on the current direction of the target (Distance on great circles). In case the distance is relatively small, the estimation will be accurate.
- Convert the current and the predicted lat, long coordinates of the target to 3D space.
- Create vector from coordinates with subtraction.
- Normalize and rescale the vector with the real target velocity

In [3]:
# The following few lines performs this calculation: 
from pygeodesy.ellipsoidalVincenty import LatLon

target_dir = 0 # North direction
target_speed = 10 #[meter/sec]

target_latlon       = LatLon(target_lat, target_lon) # Target initial coordinate
speed_vector_latlon = target_latlon.destination(1, target_dir)  # Target coordinate 1m away in the current direction
speed_vector_ecef   = ecef_converter.forward(speed_vector_latlon.lat, 
                                             speed_vector_latlon.lon, 
                                             target_ele)
speed_vector_ecef   = np.array([speed_vector_ecef[0],speed_vector_ecef[1], speed_vector_ecef[2]])
speed_vector        = speed_vector_ecef-target_ecef # Create vector in Cartesian space
speed_vector       /= np.sqrt(np.sum(np.abs(speed_vector)**2)) # Normalize
speed_vector       *= target_speed

print("Speed vector X, Y Z coordinates", speed_vector)

Speed vector X, Y Z coordinates [-6.96949954 -2.41043298  6.75395358]


The Doppler shift is given by the variation rate of the phase delay along the full signal path.
The illuminator to target distance changes to $Rt - v_i t$ (during $t$ time), while the target to receiver distance will be $R_r-v_r t$ $t$ times later. $v_i$ and $v_r$ are the speed of the target, in the direction of the illuminator and the receiver respectively.
In accordance with all these, the phase delay variation at the target is described with the following function:
\begin{equation}
\phi(t) = -1 \frac{2\pi}{\lambda} (R_t - v_i t - R_r + v_r t)
\end{equation}
, thus the Doppler frequency is: 
\begin{equation}
f_{Doppler}=\frac{1}{2 \pi} \frac{d\phi(t)}{dt}=\frac{v_i +v_r}{\lambda}
\end{equation}
To get $v_i$ and $v_r$ we can calculate the projection of the speed vector onto the illuminator to target and onto the target to receiver vectors. 
\begin{eqnarray}
v_i = \textbf{v}(\textbf{t}-\textbf{i})  \\
v_r= \textbf{v}(\textbf{t}-\textbf{r})
\end{eqnarray}

In [6]:
# As a summary the bistatic Doppler frequency can be calculated with the following script:

wavelength = 0.5 # [m] @ 600 MHz

# Generate target to IoO vector
target_to_ioo_vector  = (target_ecef-ioo_ecef)/Rt
    
# Generate target to radar vector
target_to_radar_vector = (target_ecef-radar_ecef)/Rr
    
# Calculate target Doppler
fD = -1*((np.sum(speed_vector*target_to_ioo_vector)) + (np.sum(speed_vector*target_to_radar_vector)))/wavelength 

print("Calculated bistatic Doppler frequency: {:.1f} Hz".format(fD))
    

Calculated bistatic Doppler frequency: -14.7 Hz


## Bearing  

The final missing parameter of the reflection is the angle of arrival.  To obtain this we can use the following formula to calculate the initial heading from one point to another, when they are given in lattitude and longitude.[Veness2019]
\begin{equation}
\theta = atan2(cos(lat_1)*sin(lat_2)-sin(lat_1)*cos(lat_2)*cos(lon_2-lon_1),sin(lon_2-lon_1)*cos(lat_2)) 
\end{equation}

Here the first coordinate pair ($lat_1$,$lon_1$) corresponds to the receiver location, while the second ($lat_2$, $lon_2$) is the target location.After correcting this value with the actual heading of the surveillance antenna, we get the measureble direction of arrival of the target reflection.


## Applications:

### APRiL implementation:
All these transformations and calculations are integrated into a single function that can be found in the pyAPRiL library [Peto2020a]. The following small example shows how to use it:


In [5]:
radar_bearing = 0 # [deg] relative to the north direction
from pyapril.targetParamCalculator import calculate_bistatic_target_parameters
(Rb, fD, theta) = \
calculate_bistatic_target_parameters(radar_lat, 
                                     radar_lon, 
                                     radar_ele, 
                                     radar_bearing,
                                     ioo_lat, 
                                     ioo_lon, 
                                     ioo_ele,
                                     target_lat=target_lat , 
                                     target_lon=target_lon,
                                     target_ele=target_ele,
                                     target_speed=target_speed, 
                                     target_dir=target_dir,
                                     wavelength=wavelength)
print("Bistatic distance: {:.1f} m".format(Rb))
print("Bistatic Doppler frequency: {:.1f} Hz".format(fD))
print("Angle of Arrival : {:.1f} deg".format(theta))

Bistatic distance: 7236.6 m
Bistatic Doppler frequency: -14.7 Hz
Angle of Arrival : 155.9 deg


### Automated mesurement evaluation
Using the so far obtained results and implementations, one can calculate the expected bistatic range, Doppler frequency and bearing angle of the observed target from an available target track.
( A util function located in the Vega repository [Peto2020b] named "FR24_track_reproc" can be used to calculate these fields for a Vega measurement by processing standard FlightRadar24 track files)

#### Expected target zone highlight
In the knowledge of the expected target parameters we can search for target reflection on the range-Doppler map just in the right place. As sometimes it is very difficult to find the investigated target reflection this tool could be especially valuable.
The following example shows a range-Doppler map animation where the expected tatget location area ( calculated from flightRadar24 track ) is marked with a red square. You can also note, that without the highlight of expected target zone the identification of the reflection would have been really difficult even with using a CFAR algorithm.
For VEGA database compatible measurement this script is available in the repository [Peto2020b].


![](images/RD_matrix_auto_detec_animation.gif "segment")

Figure 1: Demonstration for automatic target parameter calculation, [Peto2020b], VEGAM20191219K4U0C0S5DVBT

# Rerences

[Willis2005] Nicolas J. Willis: Bistatic Radar, SciTech Publishing, 2005  

[Veness2019] Chris Veness: https://www.movable-type.co.uk/scripts/latlong.html  

[Brouwers2020] Jean Brouwers, Chris Veness, Charles Karney: Pygeodesy 20.3.2, 1 march 2020 https://github.com/mrJean1/PyGeodesy, https://pypi.org/project/PyGeodesy/  

[Peto2020a] Tamás Pető: pyAPRiL 1.6.post2 10 march 2020, https://github.com/petotamas/APRiL, https://pypi.org/project/pyAPRiL/#data  

[Peto2020b] Tamás Pető: VEGA database, 1 march 2020, https://github.com/petotamas/vega_database_tools
